In [1]:
import os
import gc 
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings(action = 'ignore')

In [ ]:
def Reduce_Memory_Usage(data = None):
    """
    根据数据集中特征的取值进行调整减少内存占用
    
    """
    
    dtypes = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    origin_mem_usage = data.memory_usage().sum()
    for col in data.columns:
        if data[col].dtype in dtypes:
            #获取该特征列的最大最小值
#             max_value = data[col].max()
#             min_value = data[col].min()
            max_value = np.max(data[col])
            min_value = np.min(data[col])
            
            if str(data[col].dtype)[:3] == 'int':
                if max_value <= np.iinfo(np.int8).max and min_value >= np.iinfo(np.int8).min:
                    data[col] = data[col].astype('int8')
                
                elif max_value <= np.iinfo(np.int16).max and min_value >= np.iinfo(np.int16).min:
                    data[col] = data[col].astype('int16')
                
                elif max_value <= np.iinfo(np.int32).max and min_value >= np.iinfo(np.int32).min:
                    data[col] = data[col].astype('int32')
                
                elif max_value <= np.iinfo(np.int64).max and min_value >= np.iinfo(np.int64).min:
                    data[col] = data[col].astype('int64')
            else:
                if max_value <= np.finfo(np.float16).max and min_value >= np.finfo(np.float16).min:
                    data[col] = data[col].astype('float16')
                    
                elif max_value <= np.finfo(np.float32).max and min_value >= np.finfo(np.float32).min:
                    data[col] = data[col].astype('float32')
                
                elif max_value <= np.finfo(np.float64).max and min_value >= np.finfo(np.float64).min:
                    data[col] = data[col].astype('float64')

    reduced_memory_usage = data.memory_usage().sum()
    reduced_sum = (origin_mem_usage - reduced_memory_usage) 
    print("origin memory usage: {}".format(origin_mem_usage / 1024**2))
    print("reduced memory usage: {}".format(reduced_memory_usage/1024**2))
    print("memory reduce ({:.1f}%)".format(reduced_sum * 100/origin_mem_usage))
    gc.collect()